#뉴스기사 크롤링 분류

In [ ]:
# 크롤러를 만들기 전 필요한 도구들을 임포트
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article

# 페이지 수, 카테고리, 날짜를 입력
def make_urllist(page_num, code, date):
  urllist= []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)

    # BeautifulSoup의 인스턴스 생성
    soup = BeautifulSoup(news.content, 'html.parser')

    # CASE 1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    # CASE 2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))

    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'출력
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

In [ ]:
url_list = []

In [ ]:
#코드 네이밍
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

In [ ]:

#- 데이터프레임을 생성
def make_data(urllist, code):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.title)

  #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류
  df = pd.DataFrame({'news': text_list})

  #- 데이터프레임의 'code' 키 아래 한글 카테고리명
  df['code'] = idx2word[str(code)]
  return df

In [ ]:
code_list = [101, 102, 103, 105]

code_list

In [ ]:
def make_total_data(page_num, code_list, date):
  df = None

  for code in code_list:
    url_list = make_urllist(page_num, code, date)
    df_temp = make_data(url_list, code)
    print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

    if df is not None:
      df = pd.concat([df, df_temp])
    else:
      df = df_temp

  return df

In [ ]:
df = make_total_data(1, code_list, 20231128)

In [ ]:
#다른 일자 데이터 추가
df = df.append(make_total_data(1, code_list, 20231123))

In [ ]:
import os

# 데이터프레임 파일을 csv 파일로 저장
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data.csv"
df.to_csv(csv_path, index=False)

if os.path.exists(csv_path):
  print('{} File Saved!'.format(csv_path))

In [ ]:
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data.csv"
df = pd.read_table(csv_path, sep=',')
df.head()

In [ ]:
# 정규 표현식을 이용해서 한글 외의 문자 제거
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

In [ ]:
#Null 값 확인
print(df.isnull().sum())

In [ ]:
# 중복된 샘플들을 제거합니다.
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "NanumGothic"

df['code'].value_counts().plot(kind = 'bar')

In [ ]:
print(df.groupby('code').size().reset_index(name = 'count'))

In [ ]:
from konlpy.tag import Mecab

tokenizer = Mecab()

#- kor_text에 csv 파일 내용 넣기
kor_text = df['news'].to_string(index=False)

#- 형태소 분석, 즉 토큰화(tokenization)를 합니다.
print(tokenizer.morphs(kor_text))

In [ ]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스', '도',  '인', '서', '올', '와', '내일’, ‘형’, ‘로’, ‘시대’, ’곳', '과', '또', '하는', '하', '는', '속' ,'제' ,'도' ,'덜' ,'본', '부터', '전', '지역', '새', '까지', ]

In [ ]:
#불용어 개수 확인
print(len(stopwords))

In [ ]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence)
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords]
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [ ]:
text_data = preprocessing(df['news'])
print(text_data[0])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
#- 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [ ]:
#- 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [ ]:
#- 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [ ]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [ ]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

#회고

처음 접하는 내용들이라 노드의 내용을 이해하는데 시간을 많이 쏟았지만
이해가 되지 않더라도 바로 하나씩 직접 실습해보면서 이해를 했으면 더 빠르게 좋은 결과를 만들어 내지 않았을까 하는 아쉬움이 많았습니다.
